<a href="https://colab.research.google.com/github/Lvisbal/ESE-CMRC/blob/main/SIREM_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
from google.colab import drive
from google.colab import files

In [14]:
drive.mount('/content/drive')
archivo_path = '/content/drive/MyDrive/ESE Clinica MRC - Contratacion 2025/Datasets/Base-informe-9000.xlsx'
data = pd.read_excel(archivo_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# Ver nombres de las columnas en el archivo cargado
print(data.columns.tolist())

['Ranking', 'NIT', 'RAZON SOCIAL', 'SUPERVISOR', 'REGIÓN', 'DEPARTAMENTO DOMICILIO', 'CIUDAD DOMICILIO', 'CIIU', 'MACROSECTOR', 'INGRESOS OPERACIONALES\n2023*', 'GANANCIA (PERDIDA) 2023', 'TOTAL ACTIVOS 2023', 'TOTAL PASIVOS 2023', 'TOTAL PATRIMONIO 2023', 'INGRESOS OPERACIONALES\n2022*', 'GANANCIA (PERDIDA)2022', 'TOTAL ACTIVOS 2022', 'TOTAL PASIVOS 2022', 'TOTAL PATRIMONIO 2022', 'GRUPO EN NIIF']


In [16]:
# Paso 1: Extraer únicamente el código del CIIU
data['CIIU'] = data['CIIU'].str.split(' - ').str[0]

# Paso 2: Definir los códigos CIIU a filtrar
ciiu_filtrados = ['S9601', 'N8010', 'I5629', 'N8129']

# Paso 3: Filtrar los datos por los códigos CIIU requeridos
data_filtrada = data[data['CIIU'].isin(ciiu_filtrados)]

In [21]:
# Paso 4: Crear el archivo Excel con cálculos por cada CIIU
with pd.ExcelWriter('indicadores_CIIU.xlsx') as writer:
    datos_generados = False  # Bandera para verificar si se generó alguna hoja

    for ciiu in ciiu_filtrados:
        # Filtrar datos por cada CIIU
        data_ciiu = data_filtrada[data_filtrada['CIIU'] == ciiu]

        if data_ciiu.empty:
            print(f"No hay datos para el CIIU {ciiu}.")
            continue

        # Calcular los totales necesarios para los indicadores
        totales = data_ciiu[['TOTAL ACTIVOS 2023', 'TOTAL PASIVOS 2023', 'GANANCIA (PERDIDA) 2023', 'TOTAL PATRIMONIO 2023']].sum()

        # Calcular indicadores financieros
        indicadores = pd.DataFrame({
            'Indicador': ['Liquidez', 'Endeudamiento', 'Capital de Trabajo', 'ROE', 'RAE'],
            'Valor': [
                totales['TOTAL ACTIVOS 2023'] / totales['TOTAL PASIVOS 2023'] if totales['TOTAL PASIVOS 2023'] != 0 else 0,
                totales['TOTAL PASIVOS 2023'] / totales['TOTAL ACTIVOS 2023'] if totales['TOTAL ACTIVOS 2023'] != 0 else 0,
                totales['TOTAL ACTIVOS 2023'] - totales['TOTAL PASIVOS 2023'],
                totales['GANANCIA (PERDIDA) 2023'] / totales['TOTAL PATRIMONIO 2023'] if totales['TOTAL PATRIMONIO 2023'] != 0 else 0,
                totales['GANANCIA (PERDIDA) 2023'] / totales['TOTAL ACTIVOS 2023'] if totales['TOTAL ACTIVOS 2023'] != 0 else 0
            ]
        })

        # Imprimir resultados en la consola
        print(f"\nResultados para CIIU {ciiu}:")
        print(indicadores)

        # Guardar los indicadores en una hoja del archivo Excel
        indicadores.to_excel(writer, sheet_name=f'CIIU_{ciiu}', index=False)
        datos_generados = True

    # Crear una hoja de control si no hubo datos para los códigos especificados
    if not datos_generados:
        pd.DataFrame({'Mensaje': ['No se encontraron datos para los CIIU especificados.']}).to_excel(writer, sheet_name='Control', index=False)

print("\nArchivo 'indicadores_CIIU.xlsx' creado con éxito.")

# Paso 5: Descargar el archivo generado
from google.colab import files
files.download('indicadores_CIIU.xlsx')


Resultados para CIIU S9601:
            Indicador         Valor
0            Liquidez  2.816766e+00
1       Endeudamiento  3.550171e-01
2  Capital de Trabajo  1.918054e+08
3                 ROE  1.278248e-01
4                 RAE  8.244482e-02

Resultados para CIIU N8010:
            Indicador         Valor
0            Liquidez  2.158728e+00
1       Endeudamiento  4.632357e-01
2  Capital de Trabajo  2.475402e+09
3                 ROE  1.438385e-01
4                 RAE  7.720737e-02

Resultados para CIIU I5629:
            Indicador         Valor
0            Liquidez  2.459094e+00
1       Endeudamiento  4.066538e-01
2  Capital de Trabajo  3.534809e+08
3                 ROE  9.362638e-02
4                 RAE  5.555286e-02

Resultados para CIIU N8129:
            Indicador         Valor
0            Liquidez  2.341755e+00
1       Endeudamiento  4.270302e-01
2  Capital de Trabajo  1.433737e+08
3                 ROE  2.736100e-02
4                 RAE  1.567703e-02

Archivo 'indicadore

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
files.download('indicadores_CIIU.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>